In [ ]:
#https://github.com/myaichat/wxchat/blob/google_gemma/requirements-windows.txt
#https://medium.com/@alexbuzunov/gemma-2-setting-up-and-running-inference-0d7dc430b72c

In [ ]:

!pip -q install accelerate
# !pip -q install --force-reinstall /content/transformers-4.42.0.dev0-py3-none-any.whl
!pip -q install git+https://github.com/huggingface/transformers # need to install from github
!pip install -q sentencepiece


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.3/21.3 MB 46.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:

%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass


In [ ]:

!pip show transformers


Name: transformers
Version: 4.43.0.dev0
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /usr/local/lib/python3.10/dist-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, safetensors, tokenizers, tqdm
Required-by: peft, trl


# 4-bit Quantization

In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessorList, LogitsProcessor
import torch
import time

class StreamProcessor(LogitsProcessor):
    def __init__(self, tokenizer, model):
        self.tokenizer = tokenizer
        self.model = model
        self.generated_text = ""
        print("\nStart:", time.time() - start)
        self.start= time.time()

    def __call__(self, input_ids, scores):
        generated_token_id = torch.argmax(scores, dim=-1)
        generated_token = self.tokenizer.decode(generated_token_id)
        self.generated_text += generated_token
        print(generated_token, end='', flush=True)
        return scores

start = time.time()
model_id="google/gemma-2-9b-it"


In [ ]:

from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

quantization_config_2 = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)



In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config_2,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    cache_dir="./cache",
    trust_remote_code=True #,
    #low_cpu_mem_usage=True
)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/40.6k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.24M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/636 [00:00<?, ?B/s]

...

model-00002-of-00004.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/3.67G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

In [ ]:

def generate_text(prompt, max_length=50):
  """Generates text using quantized Gemma 2 in zero-shot mode."""

  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
  outputs = model.generate(**inputs, max_length=max_length)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return generated_text


In [ ]:

import time
# Start the timer
start = time.time()
# Example usage:
prompt = "Describe the tallest tower in the world"
generated_story = generate_text(prompt)
print(generated_story)


# Print the time taken
print("\nTime:", time.time() - start)

# 52s        No quantization
# 16s        4 bit quantization
# Time: Time: 8.394475936889648        4 bit quantization

Describe the tallest tower in the world.

The tallest tower in the world is the **Burj Khalifa**, located in Dubai, United Arab Emirates. 

Here are some key facts about it:

* **Height:** 828

Time: 8.394475936889648


In [ ]:


import time
# Start the timer
start = time.time()
# Example usage:
prompt = "Write me a poem about Machine Learning."
generated_story = generate_text(prompt)
print(generated_story)


# Print the time taken
print("\nTime:", time.time() - start)

# 52s        No quantization
# 16s        4 bit quantization
# Time: 8.663780927658081         4 bit quantization


Write me a poem about Machine Learning.

A mind of silicon, a heart of code,
Learning patterns, a path to be trod.
From data's depths, it seeks to discern,
Connections unseen, a knowledge to learn

Time: 8.663780927658081


In [ ]:

import time
# Start the timer
start = time.time()
# Example usage:
prompt = "Describe the tallest tower in the India"
generated_story = generate_text(prompt)
print(generated_story)


# Print the time taken
print("\nTime:", time.time() - start)

# 52s        No quantization
# 16s        4 bit quantization
# Time: 7.178032159805298       4 bit quantization


Describe the tallest tower in the India.

The tallest tower in India is the **Qutub Minar**, located in Delhi. 

Here are some details about it:

* **Height:** 73 meters (240

Time: 7.178032159805298


In [ ]:

import time
# Start the timer
start = time.time()
# Example usage:
prompt = "Describe the tallest tower in the Bangladesh"
generated_story = generate_text(prompt)
print(generated_story)


# Print the time taken
print("\nTime:", time.time() - start)

# 52s        No quantization
# 16s        4 bit quantization
# Time: Time: 8.431696653366089      4 bit quantization


Describe the tallest tower in the Bangladesh.

The tallest tower in Bangladesh is the **Bangabandhu Satellite Communication Tower**. 

Here are some details about it:

* **Height:** 151 meters (495 feet)

Time: 8.431696653366089


In [ ]:
start = time.time()
input_text = "Write me a poem about Machine Learning."
input_ids = tokenizer(input_text, return_tensors="pt").to("cuda")

logits_processor = LogitsProcessorList()
logits_processor.append(StreamProcessor(tokenizer, model))

outputs = model.generate(
    **input_ids,
    logits_processor=logits_processor,
    max_new_tokens=100,  # limit the number of new tokens to generate
    do_sample=True,  # disable sampling to get deterministic output
    temperature=1.0,
    top_p=0.95,      # Nucleus sampling
    top_k=50         # Limiting to top_k choices
)
print(outputs)
print("\nStreaming:", time.time() - logits_processor[0].start)
print("\nTotal:", time.time() - start)

#Total: 26.818603992462158


Start: 0.0159604549407959


A mind of data, a symphony of code,
Machine algorithms emerge, the unknown.
Algorithms learn, a tireless refrain,
To learn from the, to be the test.

Input the numbers, the text, the sound,
Let the machine, ignite its light.
It sifts and sorts, it weighs and sees,
Un web world, it knowledge transcends.

From pixels to prose, from starts to see,
The unseen, a you and for me.tensor([[     2,   5559,    682,    476,  19592,   1105,  13403,  14715, 235265,
            109, 235280,  11877,    576,   1423, 235269,    476, 113598,    576,
           3409, 235269,    108,   6006,  12136,  40594,    774,    573, 157855,
         235265,    108, 139972,   2561, 235269,    476, 185614,  18599, 235269,
            108,   1469,   3918,    774,  32733, 235269,    577,   2307,    573,
           2121, 235265,    109,   3678,    573,   5968, 235269,    573,   2793,
         235269,    573,  12559, 235269,    108,  30136,    573,   6479, 235269,
         121144,   1277,   2

In [ ]:

def generate_text(prompt, max_length=50):
  """Generates text using quantized Gemma 2 in zero-shot mode."""

  inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
  outputs = model.generate(**inputs, max_length=max_length)
  generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
  return generated_text
import time
# Start the timer
start = time.time()
# Example usage:
prompt = "Describe the tallest tower in the world"
generated_story = generate_text(prompt)
print(generated_story)


# Print the time taken
print("\nTime:", time.time() - start)

# 52s        No quantization
# 16s        4 bit quantization
# Time: 7.6485114097595215          4 bit quantization

In [ ]:
import time
# Start the timer
start = time.time()
# Example usage:
prompt = "Describe the tallest tower in the world"
generated_story = generate_text(prompt)
print(generated_story)


# Print the time taken
print("\nTime:", time.time() - start)

# 52s        No quantization
# 16s        4 bit quantization
# Time: 7.6485114097595215          4 bit quantization

Describe the tallest tower in the world.

The tallest tower in the world is the **Burj Khalifa**, located in Dubai, United Arab Emirates. 

Here are some key facts about it:

* **Height:** 828

Time: 7.320498704910278
